In [1]:
!pip install julep

In [95]:
import os

import yaml
from julep import Julep

api_key = os.getenv("JULEP_API_KEY")
julep = Julep(api_key=api_key, environment="dev")

In [86]:
agent = julep.agents.create(
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="gpt-4o",
    default_settings={"temperature": 0.2},
)

In [96]:
agent.id

'2790657c-8378-4c5b-a60b-08b27e8ed7cf'

In [103]:
mailgun_password = os.getenv("MAILGUN_PASSWORD")

task_def = yaml.safe_load(f"""
name: Julep Email Assistant

input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

tools:
- name: send_email
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {mailgun_password}
      port: 587
      user: postmaster@email.julep.ai

- name: search_docs
  system:
    resource: agent
    subresource: doc
    operation: search
    
main:
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ _.from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ _.subject }}}}

      {{{{ _.body }}}}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.

  unwrap: true

- tool: search_docs
  arguments:
    agent_id: "'{agent.id}'"
    text: _
    
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ inputs[0].from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ inputs[0].subject }}}}

      {{{{ inputs[0].body }}}}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{% for doc in _.docs %}}
      {{% for snippet in doc.snippets %}}
        {{{{ snippet.content }}}}
      {{% endfor %}}
    {{% endfor %}}
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.
  response_format:
    type: json_object
    
  unwrap: true
  
- evaluate:
    subject: "load_json(_.split('```json')[1].split('```')[0])['subject']"
    body: "load_json(_.split('```json')[1].split('```')[0])['body']"
    
- tool: send_email
  arguments:
    body: _.body
    from: "'postmaster@email.julep.ai'"
    subject: _.subject
    to: inputs[0].from
""")

In [104]:
task = julep.tasks.create(
    agent_id=agent.id,
    **task_def,
)

In [105]:
task.id

'614a7f59-6b43-4887-a6dd-1a75e5516025'

In [106]:
execution = julep.executions.create(
    task_id=task.id,
    input={"from": "diwank@julep.ai", "to": "help@agents.new", "subject": "what's up", "body": "sup"},
)

In [108]:
julep.executions.get(execution.id)

Execution(id='4205dda5-2afb-4f2f-8538-cbbf3e0c19f3', created_at=datetime.datetime(2024, 10, 7, 13, 1, 6, 431430, tzinfo=datetime.timezone.utc), input={'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up", 'to': 'help@agents.new'}, status='succeeded', task_id='614a7f59-6b43-4887-a6dd-1a75e5516025', updated_at=datetime.datetime(2024, 10, 7, 13, 1, 8, 518972, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={'body': 'Hi there! How can I assist you today? If you have any questions or need help with Julep, feel free to let me know!', 'subject': 'Hello!'})

In [20]:
# julep.executions.transitions.stream(execution_id=execution.id)

KeyboardInterrupt: 